<a href="https://colab.research.google.com/github/haydenclose/Cloud_based_Oil_Detection/blob/main/Cloud_Based_Oil_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloud Based Analysis of Oil Spills from Wrecks
This is a workflow on how to use Google Earth Engine (GEE) with Google Colab to identify and detect oil from shipwreck

Created By Hayden Close


##1 GammaMap Speckle filter

This code defines three functions: `powerToDb`, `dbToPower`, and `gammaMap`. These functions are used to perform gamma map filtering on a Sentinel-1 image to reduce speckle noise.

The `powerToDb` function takes an image in power scale as input and converts it to decibel (dB) scale using the formula `10 * log10(img)`. The `dbToPower` function performs the inverse operation, converting an image in dB scale back to power scale using the formula `10^(img/10)`.

The `gammaMap` function takes an image in dB scale as input and performs gamma map filtering on it to reduce speckle noise. The function first converts the image from dB to power scale using the `dbToPower` function. It then calculates the mean and variance of the image within a square kernel of size `ksize` using the `reduceNeighborhood` method. The kernel is defined using an ee.Kernel.fixed object with weights set to 1.

The function then calculates a “pure speckle” threshold based on the equivalent number of looks (`enl`) parameter. If the coefficient of variation (`ci`) within the kernel is less than or equal to this threshold (`cu`), the function returns the simple mean of the pixel values within the kernel.

If `ci` is greater than `cu` but less than a maximum value (`cmax`), the function calculates a filtered value for each pixel using a refined Lee filter. This filtered value is calculated using a complex formula that involves several intermediate variables (`alpha`, `b`, `d`, and `f`). The final result is obtained by dividing `b * mean + sqrt(d)` by `alpha * 2`.

If `ci` is greater than or equal to `cmax`, the function does not filter the pixel value at all. Instead, it returns the original pixel value from the input image.

The final result is obtained by combining these three cases into a single image using the ee.`ImageCollection.reduce` method with the `ee.Reducer.firstNonNull` reducer. This produces an image where pixels with low speckle noise are replaced by their mean value, pixels with moderate speckle noise are replaced by their filtered value, and pixels with high speckle noise are left unchanged.

The code is quite complex so not fully annotated below but more information can be found from the source of the code [Mygeoblog.com](https://mygeoblog.com/2021/01/21/sentinel-1-speckle-filter-gamma-map/)


In [ ]:
# Implementation by Andreas Vollrath (ESA), inspired by Johannes Reiche (Wageningen)
def powerToDb(img):
  return ee.Image(10).multiply(img.log10())
def dbToPower(img):
  return ee.Image(10).pow(img.divide(10))
def gammaMap(img):
  ksize = 3
  enl = 5
  bandNames = img.bandNames()

  # Convert image from dB to natural values
  nat_img = dbToPower(img)

  # Square kernel, ksize should be odd (typically 3, 5 or 7)
  weights = ee.List.repeat(ee.List.repeat(1,ksize),ksize)

  # ~~(ksize/2) does integer division in JavaScript
  kernel = ee.Kernel.fixed(ksize,ksize, weights, math.floor(ksize/2), math.floor(ksize/2), False)

  # Get mean and variance
  mean = nat_img.reduceNeighborhood(ee.Reducer.mean(), kernel)
  variance = nat_img.reduceNeighborhood(ee.Reducer.variance(), kernel)

  # "Pure speckle" threshold
  ci = variance.sqrt().divide(mean);  # square root of inverse of enl

  # If ci <= cu, the kernel lies in a "pure speckle" area -> return simple mean
  cu = 1.0/math.sqrt(enl)

  # If cu < ci < cmax the kernel lies in the low textured speckle area -> return the filtered value
  cmax = math.sqrt(2.0) * cu

  alpha = ee.Image(1.0 + cu*cu).divide(ci.multiply(ci).subtract(cu*cu))
  b = alpha.subtract(enl + 1.0)
  d = mean.multiply(mean).multiply(b).multiply(b).add(alpha.multiply(mean).multiply(nat_img).multiply(4.0*enl))
  f = b.multiply(mean).add(d.sqrt()).divide(alpha.multiply(2.0))

  caster = ee.Dictionary.fromLists(bandNames,ee.List.repeat('float',3))
  img1 = powerToDb(mean.updateMask(ci.lte(cu))).rename(bandNames).cast(caster)
  img2 = powerToDb(f.updateMask(ci.gt(cu)).updateMask(ci.lt(cmax))).rename(bandNames).cast(caster)
  img3 = img.updateMask(ci.gte(cmax)).rename(bandNames).cast(caster)

  # If ci > cmax do not filter at all (i.e. we don't do anything, other then masking)
  result = ee.ImageCollection([img1,img2,img3]) \
    .reduce(ee.Reducer.firstNonNull()).rename(bandNames)

  # Compose a 3 band image with the mean filtered "pure speckle", the "low textured" filtered and the unfiltered portions
  return result


## 2 Setup function to set parameters
Creates widgets such  as drop down boxes, toggle buttons and sliders to fine tune the mapping

In [ ]:
def SetUp():                                                                                        # Function for setup widgets
  global WRKdropdown, StartDate, EndDate, orbit, SatelliteNo, max_cloud_cover, Dilation, PixelFilter# Variables to make available                                                     # Need this as otherwise it looks internally of the function for values
  Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.4.xls')                          # Load in the wreck data
  WRKdropdown = widgets.Dropdown(options=list(Wrecks.Wreck_ID),                                     # List of the wrecks from our data frame
                            value='HMS REPULSE',                                                    # Default value, here the repulse as one using as an example
                            description='Wreck:')                                                   # Descriptor in front of dropdown
  StartDate = widgets.DatePicker(description='Start Date')                                          # Setup the calendar to pick the start date to investigate
  EndDate = widgets.DatePicker(description='End Date')                                              # Setup the calendar to pick the end date to investigate
  orbit = widgets.RadioButtons(options=list(['ASCENDING','DESCENDING','BOTH']),                     # List of the wrecks from our data frame
                            value='DESCENDING',                                                     # Default value, here the repulse as one using as an example
                            description='Sentinel-1 Orbit Pass:')                                   # Descriptor in front of dropdown
  SatelliteNo = widgets.RadioButtons(options=list(['SENTINEL-1','SENTINEL-2', 'BOTH']),             # List of the wrecks from our data frame
                            value='BOTH',                                                           # Default value, here the repulse as one using as an example
                            description='Satellites:')                                              # Descriptor in front of dropdown
  max_cloud_cover = widgets.IntSlider(value = 100,                                                  # Default value
                                      min = 0,                                                      # Minimum value
                                      max = 100,                                                    # Maximum value
                                      step = 1,                                                     # Incremental step
                                      description = 'Max cloud%',                                   # Description of the slider
                                      readout = True,                                               # Provides the value on screen
                                      orientation = 'horizontal',                                   # Can be horizontal or vertical
                                      layout=widgets.Layout(width='400px'))                         # Size of widget
  Dilation = widgets.IntSlider(value = 3,                                                           # Value to dilate each pixel by
                                      min = 0,                                                      # Min dilation value
                                      max = 30,                                                     # Maximum dilation value
                                      step = 1,                                                     # Incremental step
                                      description = 'Dilation factor',                              # Description of the slider
                                      readout = True,                                               # Prints the slider value
                                      orientation = 'horizontal',                                   # Can be horizontal or vertical
                                      layout=widgets.Layout(width='400px'))                         # Size of widget
  PixelFilter = widgets.IntSlider(value = 250,                                                      # Removes polygons less than this size
                                      min = 0,                                                      # Min pixel filter size
                                      max = 1000,                                                   # Max pixel filter size
                                      step = 1,                                                     # Incremental step
                                      description = 'Pixel Filter',                                 # Description of the slider
                                      readout = True,                                               # Prints the slider value
                                      orientation = 'horizontal',                                   # Can be horizontal or vertical
                                      layout=widgets.Layout(width='400px'))                         # Size of widget
  return(display(WRKdropdown,StartDate,EndDate, SatelliteNo,orbit, max_cloud_cover, Dilation, PixelFilter)) # Returns the widgets

## 3 Functions to Map Combining Sentinel-1 and sentinel-2
see comments for details. Returns the geeMap with the first image in the image collection either S1 or S2 and with widgets to move forward or backwards in the imagecollection along with a slider to adjust the threshold


In [ ]:
def addDate(feature):                                                                               # Function to add date to the individual polygons
  return feature.set({'Date': date_string,'ImgNo': n, 'PolyNo': m })                                # Adds the date to the feature part of each polygon in the feature collection

def add_S1Gamma_date(image):                                                                        # Extract the date for for the S1 Gamma corrected images as some metadata removed during the processing
 year = ee.Image(image).getString('system:index').slice(-50, 21)                                    # Select out the year
 month = ee.Image(image).getString('system:index').slice(-46, 23)                                   # Select out the month
 day = ee.Image(image).getString('system:index').slice(-44, 25)                                     # Select out the day
 Date = ee.String(year.cat('-').cat(month).cat('-').cat(day))                                       # Format to date format e.g. 2023_01_01
 Satellite = ee.Image(image).getString('system:index').slice(0, 3)                                  # Select the name of the satellite to filter
 return image.set('Date', Date, 'Satellite',Satellite)                                              # Return the image with the added data

def add_S1_date(image):                                                                             # Select out the date and metadata for S1 imagery
    date = image.date().format('YYYY-MM-dd')                                                        # Get the date
    Satellite = ee.Image(image).getString('familyName')                                             # Get the satellite name
    platform_number = ee.Image(image).getString('platform_number')                                  # Get the name i.e. 1a or 1b
    Satellite = Satellite.cat(platform_number)                                                      # Combine these together
    return image.set('Date', date, 'Satellite',Satellite )                                          # Return the image with the added data

def add_S2_date(image):                                                                             # Select out the date and metadata for S2 imagery
    date = image.date().format('YYYY-MM-dd')                                                        # Get the date
    Satellite = ee.Image(image).getString('SPACECRAFT_NAME')                                        # Get the satellite name
    return image.set('Date', date, 'Satellite',Satellite )                                          # Return the image with the added data

def update_slider(img):                                                                             # Function to update the threshold slider
  if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B': # If S1 do below changes
       Slider_value=[-30,-23.5]                                                                     # Default slider settings
       Slider_min=-30.0                                                                             # Min value
       Slider_max=-15.0                                                                             # Max value
       Slider_step=0.5                                                                              # Step change in slider
       ChangeThreshformat='.1f'                                                                     # Change format to 1 decimal place
  else:                                                                                             # If S2 then apply below
       Slider_value=[2350,3000]                                                                     # Default slider settings
       Slider_min=1500                                                                              # Min value
       Slider_max=7000                                                                              # Max value
       Slider_step=25                                                                               # Step change in slider
       ChangeThreshformat='d'                                                                       # Change format to integer
  if Slider_max < ChangeThresh.min:                                                                 # slider doesnt like it if the Max less than min
     ChangeThresh.min = Slider_min                                                                  # Changes the slider min value
     ChangeThresh.max = Slider_max                                                                  # Changes the slider max value
  else:                                                                                             # If max greater than min then...
     ChangeThresh.max = Slider_max                                                                  # Changes the slider max value
     ChangeThresh.min = Slider_min                                                                  # Changes the slider min value
  ChangeThresh.step = Slider_step                                                                   # Update slider step value
  ChangeThresh.value = Slider_value                                                                 # Update slider default value
  ChangeThresh.readout_format=ChangeThreshformat                                                    # Change formatting of display

def OilMapping():                                                                                   # Function to display map
 global ImgList, Oil_Polygons, Datatable, n, m, date_string, output_widget, Oil_Polygonstmp,img, ChangeThreshWidget, ChangeThresh, ImgListGM, img_B_OSI, MultiMap# Need to make sure it brings the data from outside the function
 Oil_Polygons = ee.FeatureCollection([])                                                            # Need an empty FeatureCollection to Append our polygons to, add here so resets
 Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.4.xls')                           # Load in the wreck list
 Lat = pd.to_numeric(Wrecks.loc[Wrecks['Wreck_ID'] == WRKdropdown.value]['Latitude'])               # Get the latitude of selected wreck
 Lon = Wrecks.loc[Wrecks['Wreck_ID'] == WRKdropdown.value]['Longitude']                             # Get the longitude of selected wreck
 geom = ee.Geometry.Point(Lon.iloc[0],Lat.iloc[0]);                                                 # Loction of chosen wreck

 if SatelliteNo.value == 'SENTINEL-1' or SatelliteNo.value == 'BOTH':                               # If to see if create feature collection with S1 or both
    S1ImgCol = (ee.ImageCollection('COPERNICUS/S1_GRD').                                            # Selects the Sentinel 1 image collection
      filterDate(str(StartDate.value), str(EndDate.value)).                                         # Selects only the dates from time period chosen above
      filterMetadata('instrumentMode', 'equals', 'IW').                                             # Selects the instrument mode that we want
      filter(ee.Filter.eq('orbitProperties_pass', str(orbit.value))).                               # Selects the orbit path we want
      filterBounds(geom))                                                                           # Selects only images that our wreck is contained within
    S1ImgCol = S1ImgCol.filter(ee.Filter.listContains('system:band_names', 'VV'))                   # Only selects VV band as missing in some images
    S1ImgCol = S1ImgCol.map(add_S1_date)                                                            # Adds the image date to image metadata in easy way to read
    ImgCol_gammaMap = S1ImgCol.map(gammaMap)                                                        # Process the ImageCollection through the gammaMap algorithm
    ImgCol_gammaMap = ImgCol_gammaMap.map(add_S1Gamma_date)                                         # Gamma function removes alot of the metadata so add date back in

 if SatelliteNo.value == 'SENTINEL-2' or SatelliteNo.value == 'BOTH':                               # If to see if create feature collection with S2 or both
  S2ImgCol = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED').                                       # Selects the Sentinel 2 image collection
     filterDate(str(StartDate.value), str(EndDate.value)).                                          # Selects only the dates from time period chosen above
     filterBounds(geom).                                                                            # Selects only images that our wreck is contained within
     filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', max_cloud_cover.value))                 # Filter image collection by cloud cover
  S2ImgCol = S2ImgCol.map(add_S2_date)                                                              # Add the image date to the metadata in an easy way to read

 if SatelliteNo.value == 'BOTH':                                                                    # If both S1 and S2 combine collections
   ImageCol = S1ImgCol.merge(S2ImgCol)                                                              # Merge ImageCollections, note raw S1
   ImageCol = ImageCol.sort("Date")                                                                 # Order images by Date
   ImgList = ee.ImageCollection(ImageCol).toList(99999)                                             # Creates a list of the images to select from
   ImageColGM = ImgCol_gammaMap.merge(S2ImgCol)                                                     # Merge ImageCollections, note gammaS1
   ImageColGM = ImageColGM.sort("Date")                                                             # Order images by Date
   ImgListGM = ee.ImageCollection(ImageColGM).toList(99999)                                         # Creates a list of the d images to select from
 elif SatelliteNo.value == 'SENTINEL-1':                                                            # Create collections based just on S1
   ImageCol = S1ImgCol                                                                              # ImageCollection = S1
   ImgListGM = ee.ImageCollection(ImgCol_gammaMap).toList(99999)                                    # Creates a list of the gammacorrected images to select from
 elif  SatelliteNo.value == 'SENTINEL-2':                                                           # Create collections based just on S2
     ImageCol = S2ImgCol                                                                            # ImageCollection = S2
 ImgList = ee.ImageCollection(ImageCol).toList(99999)                                               # Creates a list of the images to select from
 Datatable = pd.DataFrame(columns = ['Wreck_Name', 'Date', 'Oil_Area_m2','Low_Threshold','High_Threshold', 'Satellite','Comment'])# Empty dataframe for polygon oil spill area
 MultiMap = geemap.Map()                                                                            # Base map
 MultiMap.centerObject(geom, 10)                                                                    # Center the map on the wreck
 MultiMap.add_points_from_xy(Wrecks, x="Longitude", y="Latitude")                                   # Add wreck locations
 m = 0                                                                                              # Number to assign polygon number
 n = 0                                                                                              # Used to add or subtract to change the image
 img = ee.Image(ee.List(ImgList).get(n))                                                            # Get the image from the image list to display the first image
 if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B': # If S1 use below parmaeters
  img = ee.Image(ee.List(ImgList).get(n)).select('VV')                                              # Selects the VV band
  img_params = {'bands':'VV', 'min':-25, 'max':5}                                                   # Display setting for the VV band
 else: img_params = {'min': 0,'max': 3000,'bands': ['B4','B3','B2']}                                # Otherwise select the RGB bands
 IMgdate = img.date()                                                                               # Get the acquisition date of the image
 date_string = IMgdate.format('YYYY-MM-dd').getInfo()                                               # Format the date as a string
 MultiMap.addLayer(img, img_params, 'Satellite Image',True)                                         # Add the image to the map

 output_widget = widgets.Output(layout={'border': '1px solid black'})                               # Set up widget for adding date
 output_control = ipyleaflet.WidgetControl(widget=output_widget, position='bottomright')            # Method to add widget
 MultiMap.add_control(output_control)                                                               # Adds the widget
 with output_widget:                                                                                # The date widget update
   print(date_string)                                                                               # Prints the text to the geemap
## Set up of the slider
 if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B': # If S1
       Slider_value=[-30,-23.5]                                                                     # Default
       Slider_min=-30.0                                                                             # Min value
       Slider_max=-15.0                                                                             # Max value
       Slider_step=0.5                                                                               # Step change in slider
       ChangeThreshformat='.1f'                                                                     # Add 1 decimal place
 else:                                                                                              # Else if S2 use below settings
       Slider_value=[2350,3000]                                                                     # Default
       Slider_min=1500                                                                              # Min value
       Slider_max=7000                                                                              # Max value
       Slider_step=25                                                                               # Step change in slider
       ChangeThreshformat='d'                                                                       # Change format to integer

 ChangeThresh =  widgets.FloatRangeSlider(                                                          # Defines the Oil threshold slider
       value=Slider_value,                                                                          # Default
       min=Slider_min,                                                                              # Min value
       max=Slider_max,                                                                              # Max value
       step=Slider_step,                                                                            # interval step value
       description='Thresh:',                                                                       # Description
       orientation='horizontal',                                                                    # Horizontal or Vertical
       readout=True,                                                                                # Prints value
       readout_format=ChangeThreshformat,                                                           # Format of value (1 decimal place or integer)
       layout=widgets.Layout(width='400px'))                                                        # Width of the slider
 ChangeThreshOutput = widgets.Output()                                                              # Output to display for the widget
 ChangeThreshWidget = ipyleaflet.WidgetControl(widget=ChangeThresh, position='bottomleft')          # Method to add to the map
 MultiMap.add_control(ChangeThreshWidget)                                                           # Add the control of widget to map
#################################################################################
 DetectOilButton = widgets.Button(description="Detect potential oil")                               # Button widget to add to the map
 DetectOilButtonOutput = widgets.Output()                                                           # Output to display for the widget
 DetectOilButtonWidget = ipyleaflet.WidgetControl(widget=DetectOilButton, position='bottomright')   # Method to add to the map
 MultiMap.add_control(DetectOilButtonWidget)                                                        # Adds to the map

 def on_Detect_button_clicked(b):                                                                   # Define a function for what happens on button click
   global Oil_Polygonstmp, n                                                                        # Need to make sure it brings the data from outside the function
   with DetectOilButtonOutput:                                                                      # Below happens when button clicked
     AoI = ee.FeatureCollection(MultiMap.draw_features)                                             # Extracts the drawn polygon from the map
     img = ee.Image(ee.List(ImgList).get(n)).clip(AoI)                                              # Gets the next image in the list and clips by AoI
     if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B': # If S1 selected does below
       img = ee.Image(ee.List(ImgListGM).get(n)).select('VV').clip(AoI)                               # Gets the next S1 image in the list and clips by AoI
       Oil = (img.lt(ChangeThresh.value[1]).selfMask().rename('Pixels'))                            # Select pixels lower than higher limit of threshold
       Oil  = (Oil.gt(ChangeThresh.value[0]).selfMask().rename('Pixels'))                           # Select pixels lower than lower limit of threshold
     else:                                                                                          # If S2 then apply following
       rB = img.expression('(b("B3") + b("B2"))',                                                   # To create the the red, add the bands b2 and b3
          {'b': img.select(["B3", "B2"])}).rename('rB')                                             # Create new band with expression and call it rB
       gB = img.expression('(b("B3") + b("B4")) / b("B2")',                                         # To create the green, add band b3 to b4/b2
         {'b': img.select(["B3", "B4", "B2"])}).rename('gB')                                        # Create new band with expression and call it gB
       bB = img.expression('(b("B6") + b("B7")) / b("B5")',                                         # To create the green, add band b6 to b7/b5
         {'b': img.select(["B6", "B7", "B5"])}).rename('bB')                                        # Create new band with expression and call it bB
       img_B_OSI = ee.Image([rB, gB, bB])                                                           # Create a new image based on the bands
       Oil = img_B_OSI.select('rB').gt(ChangeThresh.value[0]).And(img_B_OSI.select('rB').lt(ChangeThresh.value[1])).selfMask() # Select pixels based on threshold
     OilDIL = Oil.focal_max(kernel=ee.Kernel.circle(radius=Dilation.value))                         # Dilate the pixels to grow regions

     #MultiMap.addLayer(Oil, {'palette': 'FF0000'}, ('Pixels'))                                     # REMOVED. Displays those with pixels value 23.5 of non dilated
     Oil_Polygonstmp = OilDIL.reduceToVectors(geometry = AoI,                                       # Polygon extracted from the map
                             scale = 10,                                                            # Resolution of data in meters
                             geometryInNativeProjection =True,                                      # Use the image projection
                             maxPixels = 1e10,                                                      # Max pixels GEE will deal with
                             eightConnected =True)                                                  # Polgons will count as connect if at a diagonal
     Oil_Polygonstmp = Oil_Polygonstmp.filter(ee.Filter.gt('count',PixelFilter.value))              # Filters the polgons whic have more than 25 pixels
     Oil_Polygonstmp = Oil_Polygonstmp.map(addDate)                                                 # Adds image date to the polygons
     empty = ee.Image().byte()                                                                      # Empty ee.image to append the polygons ot
     outline = empty.paint(featureCollection= Oil_Polygonstmp,color= 'ImgNo')                       # Applies colour to the outline and fills it
     MultiMap.addLayer(outline.randomVisualizer(),name ='Temp Oil Spill Polygons')                  # Add to the map
 DetectOilButton.on_click(on_Detect_button_clicked)                                                 # Makes changes when clicked
########################################################################################################
 NextButton = widgets.Button(description="Next image")                                              # Button widget to add to the map
 NextButtonOutput = widgets.Output()                                                                # Output to display for the widget
 NextButtonWidget = ipyleaflet.WidgetControl(widget=NextButton, position='bottomright')             # Method to add to the map
 MultiMap.add_control(NextButtonWidget)                                                             # Adds to the map

 def on_Nxt_button_clicked(b):                                                                      # Define a function for what happens on button click
   global n, m, date_string, Datatable, Oil_Polygons, output_widget, Oil_Polygonstmp,img, ChangeThresh # Need this as otherwise it looks internally of the function for values
   spillarea = 0                                                                                    # Default spill area zero
   MultiMap.remove_last_drawn()                                                                     # Removes last drawn polygon
   output_widget.clear_output()                                                                     # Remove display of the date of last image
   MultiMap.remove_layer(MultiMap.find_layer('Oil Spill Polygons'))                                 # Removes the spills polygons so updated version can be added
   MultiMap.remove_layer(MultiMap.find_layer('Pixels'))                                             # Remove the Pixels layer identified from thresold
   MultiMap.remove_layer(MultiMap.find_layer('Satellite Image'))                                    # Removes the previous sat image
   MultiMap.remove_layer(MultiMap.find_layer('Temp Oil Spill Polygons'))                            # Removes the Temp oil spill layer
   if 'Oil_Polygonstmp' in globals():                                                               # Do the following if the tmp oil polygon layer exists
      m+=1                                                                                          # Adds one to the polygon count
      Oil_Polygons = Oil_Polygons.merge(Oil_Polygonstmp)                                            # Adds polygons from last detection
      spillarea = Oil_Polygonstmp.geometry().area(maxError = 1).getInfo()                           # Gets the area of the total oil spill of the last image
      del Oil_Polygonstmp                                                                           # Remove so cant keep getting appended to the oi layer
   Sat = ee.Image(img).getString('Satellite').getInfo()                                             # Get the satellite name
   data =pd.DataFrame({'Wreck_Name': [WRKdropdown.value],                                           # Append the data and polygon data, get the Wreckname
                       'Date':[date_string],                                                        # Append date
                       'Oil_Area_m2': [spillarea],                                                  # Append the spill area
                       'Low_Threshold':[ChangeThresh.value[0]],                                     # Append the low threshold value
                       'High_Threshold':[ChangeThresh.value[1]],                                    # Append the high threshold value
                       'Satellite':[Sat],                                                           # Append the satellite name
                       'Comment':''})                                                               # Blank to add comment
   if date_string in Datatable['Date'].values:                                                      # Check if the date already exists in the DataFrame
    # Overwrite the existing row if already exist and changing the thresholds
    Datatable.loc[Datatable['Date'] == date_string, ['Wreck_Name', 'Date',
                                                     'Oil_Area_m2','Low_Threshold',
                                                     'High_Threshold','Satellite','Comment' ]] = [WRKdropdown.value, date_string, spillarea, ChangeThresh.value[0],ChangeThresh.value[1],Sat,'']
   else:                                                                                            # Otherwise just append
    Datatable = Datatable.append(data, ignore_index=True)                                           # Append the new row to the DataFrame
   n += 1                                                                                           # Plus one to the number in the image list
   with NextButtonOutput:                                                                           # Below happens when button clicked
     PrevDate = date_string                                                                         # Store the previous date
     img = ee.Image(ee.List(ImgList).get(n))                                                        # Get the next image in the list
     IMgdate = img.date()                                                                           # Get the acquisition date of the image
     date_string = IMgdate.format('YYYY-MM-dd').getInfo()                                           # Format the date as a string
     ## Method of skipping duplicate images
     if date_string == PrevDate:                                                                    # If image dates the same
        n+=1                                                                                        # Add one to the image list counter
        img = ee.Image(ee.List(ImgList).get(n))                                                     # Get the next image
        IMgdate = img.date()                                                                        # Get the acquisition date of the image
        date_string = IMgdate.format('YYYY-MM-dd').getInfo()                                        # Format the date as a string
     if Datatable['Date'].isin([date_string]).any():                                                # If the image date in the datatable then overwrite
        Oil_Polygons =  Oil_Polygons.filter(ee.Filter.neq('Date', date_string))                     # Removes the Polygons from that date

     update_slider(img)                                                                             # Update the slider values depending on satellite
     if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B': # If S1 then do the below
       img = ee.Image(ee.List(ImgList).get(n)).select('VV')                                         # Selects the VV band
       img_params = {'bands':'VV', 'min':-25, 'max':5}                                              # Display setting for the VV band
     else: img_params = {'min': 0,'max': 3000,'bands': ['B4','B3','B2']}                            # Else use the S2 image bands to create RGB

     output_widget = widgets.Output(layout={'border': '1px solid black'})                           # Set up widget for adding date
     output_control = ipyleaflet.WidgetControl(widget=output_widget, position='bottomright')        # Method to add widget
     MultiMap.add_control(output_control)                                                           # Adds the widget
     with output_widget:                                                                            # The date widget update
       print(date_string)                                                                           # Print it to the map
     MultiMap.addLayer(img, img_params, 'Satellite Image',True)                                     # Add the layer to the map
     empty = ee.Image().byte()                                                                      # Empty image
     PlotPolys = Oil_Polygons.filter(ee.Filter.Or(ee.Filter.eq('PolyNo',m-1),ee.Filter.eq('PolyNo',m-2),ee.Filter.eq('PolyNo',m-3))) # Only plots the last three polygons otherwise memory becomes an issue
     outline = empty.paint(featureCollection= PlotPolys,color= 'PolyNo')                            # Add the polygon and colour
     MultiMap.addLayer(outline.randomVisualizer(),name ='Oil Spill Polygons')                       # Add the oil polygons to the map
 NextButton.on_click(on_Nxt_button_clicked)                                                         # actions the function when the button clicked

############################################################################################################
 PrevButton = widgets.Button(description="Previous image")                                          # Button widget to add to the map
 PrevButtonOutput = widgets.Output()                                                                # Output to display for the widget
 PrevButtonWidget = ipyleaflet.WidgetControl(widget=PrevButton, position='bottomleft')              # Method to add to the map
 MultiMap.add_control(PrevButtonWidget)                                                             # Adds to the map

 def on_Prevbutton_clicked(b):                                                                      # Define a function for what happens on button click
   global n, m, date_string, Datatable, Oil_Polygons, output_widget, Oil_Polygonstmp,img            # Need this as otherwise it looks internally of the function for values
   if Datatable['Date'].isin([date_string]).any():                                                  # If the image date in the datatable then overwrite
        Oil_Polygons = Oil_Polygons.filter(ee.Filter.neq('Date', date_string))                      # Removes the Polygons from that date
   spillarea = 0                                                                                    # Default spill area zero
   MultiMap.remove_last_drawn()                                                                     # Removes last drawn polygon
   output_widget.clear_output()                                                                     # Remove display of the date of last image
   MultiMap.remove_layer(MultiMap.find_layer('Oil Spill Polygons'))                                 # Removes the spills polygons so updated version can be added
   MultiMap.remove_layer(MultiMap.find_layer('Pixels'))                                             # Remove the Pixels layer identified from thresold
   MultiMap.remove_layer(MultiMap.find_layer('Satellite Image'))                                    # Removes the previous sat image
   MultiMap.remove_layer(MultiMap.find_layer('Temp Oil Spill Polygons'))                            # Removes the Temp oil spill layer
   if 'Oil_Polygonstmp' in globals():                                                               # Do the following if the tmp oil polygon layer exists
      m +=1
      Oil_Polygons = Oil_Polygons.merge(Oil_Polygonstmp)                                            # Adds polygons from last detection
      spillarea = Oil_Polygonstmp.geometry().area(maxError = 1).getInfo()                           # Gets the area of the total oil spill of the last image
      del Oil_Polygonstmp                                                                           # Remove so cant keep getting appended to the oi layer
   Sat = ee.Image(img).getString('Satellite').getInfo()                                             # Get the satellite name
   data =pd.DataFrame({'Wreck_Name': [WRKdropdown.value],                                           # Append the data and polygon data, get the Wreckname
                       'Date':[date_string],                                                        # Append date
                       'Oil_Area_m2': [spillarea],                                                  # Append the spill area
                       'Low_Threshold':[ChangeThresh.value[0]],                                     # Append the low threshold value
                       'High_Threshold':[ChangeThresh.value[1]],                                    # Append the high threshold value
                       'Satellite':[Sat],                                                           # Append the satellite name
                       'Comment':''})                                                               # Blank to add comment

   if date_string in Datatable['Date'].values:                                                      # Check if the date already exists in the DataFrame
    # Overwrite the existing row if already exist and changing the thresholds
    Datatable.loc[Datatable['Date'] == date_string, ['Wreck_Name', 'Date',
                                                     'Oil_Area_m2','Low_Threshold',
                                                     'High_Threshold','Satellite','Comment' ]] = [WRKdropdown.value, date_string, spillarea, ChangeThresh.value[0],ChangeThresh.value[1],Sat,'']
   else:                                                                                            # Otherwise just append
    Datatable = Datatable.append(data, ignore_index=True)                                           # Append the new row to the DataFrame
   n -= 1                                                                                           # minus one to the number in the image list
   with PrevButtonOutput:                                                                           # Below happens when button clicked
     img = ee.Image(ee.List(ImgList).get(n))                                                        # Get the latest image
     update_slider(img)                                                                             # Update the slider
     if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B': # If S1 then do below
       img = ee.Image(ee.List(ImgList).get(n)).select('VV')                                         # Selects the VV band
       img_params = {'bands':'VV', 'min':-25, 'max':5}                                              # Display setting for the VV band
     else: img_params = {'min': 0,'max': 3000,'bands': ['B4','B3','B2']}                            # Else use the S2 image bands to create RGB
     IMgdate = img.date()                                                                           # Get the acquisition date of the image
     date_string = IMgdate.format('YYYY-MM-dd').getInfo()                                           # Format the date as a string
     output_widget = widgets.Output(layout={'border': '1px solid black'})                           # Set up widget for adding date
     output_control = ipyleaflet.WidgetControl(widget=output_widget, position='bottomright')        # Method to add widget
     MultiMap.add_control(output_control)                                                           # Adds the widget
     with output_widget:                                                                            # The date widget update
       print(date_string)                                                                           # Print date to the Geemap
     MultiMap.addLayer(img, img_params, 'Satellite Image',True)                                     # Add the layer to the map
     PlotPolys = Oil_Polygons.filter(ee.Filter.Or(ee.Filter.eq('PolyNo',m-1),ee.Filter.eq('PolyNo',m-2),ee.Filter.eq('PolyNo',m-3))) # Only Display the previous 3 polygons to save memory
     empty = ee.Image().byte()                                                                      # Empty image
     outline = empty.paint(featureCollection= PlotPolys,color= 'PolyNo')                            # Add the polygon and colour
     MultiMap.addLayer(outline.randomVisualizer(),name ='Oil Spill Polygons')                       # Add the oil polygons to the map
 PrevButton.on_click(on_Prevbutton_clicked)                                                         # Actions the function when clicked
 return MultiMap                                                                                    # Returns the geeMap

## 4 Threshold detection
The below function provides a graph of the pixel values for a line drawn on the geeMap for either S1 or S2 images

In [ ]:
def value_detector(img):
  Line = ee.FeatureCollection(MultiMap.draw_features)                                               # Extracts the line for extracting pixel vaues
  if ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(img).getString('Satellite').getInfo()  == 'SENTINEL-1B': # If S1 do the below
   img2p = ee.Image(ee.List(ImgListGM).get(n)).select('VV')                                         # Select the image VV band
   LineGraph = img2p.sample(Line, 10).aggregate_array("VV").getInfo()                               # Extract the data from image along the line
   LineGraphDF = pd.DataFrame(LineGraph, columns=['VV'])                                            # Convert to dataframe
   fig = px.line(data_frame=LineGraphDF,y = 'VV', markers = True, width=1000, height=800).update_layout( # Creates the figure with titles related S1
                 xaxis_title="Pixel number along the line", yaxis_title="band VV")
  else:                                                                                             # If S2 then do the below
   rB = img.expression('(b("B3") + b("B2"))',                                                       # To create the the red, add the bands b2 and b3
          {'b': img.select(["B3", "B2"])}).rename('rB')                                             # Create new band with expression and call it rB
   gB = img.expression('(b("B3") + b("B4")) / b("B2")',                                             # To create the green, add band b3 to b4/b2
         {'b': img.select(["B3", "B4", "B2"])}).rename('gB')                                        # Create new band with expression and call it gB
   bB = img.expression('(b("B6") + b("B7")) / b("B5")',                                             # To create the green, add band b6 to b7/b5
         {'b': img.select(["B6", "B7", "B5"])}).rename('bB')                                        # Create new band with expression and call it bB
   img_B_OSI = ee.Image([rB, gB, bB])                                                               # Create a new image based on the bands
   B_OSI_rB = img_B_OSI.sample(Line, 10).aggregate_array("rB").getInfo()                            # Extract the data from image along the line
   B_OSI_rBDF = pd.DataFrame(B_OSI_rB, columns=['rB'])                                              # Convert to dataframe
   fig = px.line(data_frame=B_OSI_rBDF,y = 'rB', markers = True, width=1000, height=800).update_layout( # Creates the figure with titles related S2
                 xaxis_title="Pixel number along the line", yaxis_title="bands B3 + B2")
  return fig.show()                                                                                 # Returns the figure


## 5 Function to display datatable with comment box
Returns a comment box that adds comments to the latest saved image data and provides the datatable. Note the function needs refreshing to see the comment.

In [ ]:
def Display_Data():
  global Comment, CommitButOutput, CommitBut                                                        # Variables needed to be added to global environment
  CommitButOutput = widgets.Output()                     # Output function for the button
  Comment= widgets.Textarea(placeholder='Add comment for image here',                               # Set up of the comment box, placeholder description
                            description='Comment:',                                                 # Text before box
                            disabled=False)                                                         # Can make it so cant edit want it editable
  CommitBut = widgets.Button(                                                                       # Button to commit text to the datatable
     value=False,                                                                                   # No value
     description='Commit comment',                                                                  # Description of button
     disabled=False,
     button_style='', # 'success', 'info', 'warning', 'danger' or ''                                # Various styles, here just have basic
     tooltip='Description',                                                                         # If hover it gives a description
     icon='check' # (FontAwesome names without the `fa-` prefix)                                    # Just want basic check icon
)
  def on_combutton_clicked(b):                                                                      # Define a function for what happens on button click
    global Datatable, n,comment                                                                     # Need to make sure it brings the data from outside the function
    with CommitButOutput:                                                                           # Below happens when button clicked
      Row = n-1                                                                                     # Want it for the previous row entry
      Datatable.iloc[Row,6] = Comment.value                                                         # Selects the correct column to add comment text
  CommitBut.on_click(on_combutton_clicked)                                                          # Actions the button press
  display(Comment,CommitBut,Datatable)                                                              # returns the comment box, button and datatable

## 6 Return the Final plot
Creates an interactive geemap with all the polygons

In [ ]:
def Plot_Polygons():
  Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.4.xls')                          # Load in the wreck list
  Lat = pd.to_numeric(Wrecks.loc[Wrecks['Wreck_ID'] == WRKdropdown.value]['Latitude'])              # Get the latitude of selected wreck
  Lon = Wrecks.loc[Wrecks['Wreck_ID'] == WRKdropdown.value]['Longitude']                            # Get the longitude of selected wreck
  geom = ee.Geometry.Point(Lon.iloc[0],Lat.iloc[0]);                                                # Point of the Wreck

  FinalMap = geemap.Map()                                                                           # Base map
  FinalMap.centerObject(geom, 10)                                                                   # Center the map on the wreck
  FinalMap.add_points_from_xy(Wrecks, x="Longitude", y="Latitude")                                  # Add wreck locations
  empty = ee.Image().byte()                                                                         # Empty image
  outline = empty.paint(featureCollection= Oil_Polygons,color= 'PolyNo')                            # Add the polygon and colour
  FinalMap.addLayer(outline.randomVisualizer(),name ='Oil Spill Polygons')                          # Add the oil polygons to the map
  return FinalMap

## 7 Timelapse creation
Most of this code was adapated from [geeMap timelapse.py](https://github.com/gee-community/geemap/blob/master/geemap/timelapse.py#L93) and [geeMap Common.py](https://github.com/gee-community/geemap/blob/master/geemap/common.py) within the timelapse module written by gisqws's awesome work! This is subsequently not highly annotated.

Second code chunk mostly my work and thus annotated.
Things to be able to tinker. The dimensions= 2900 max, FPS

In [ ]:
def get_image_thumbnail(
    ee_object,
    out_img,
    vis_params,
    dimensions=500,
    region=None,
    format="jpg",
    crs="EPSG:3857",
    timeout=300,
    proxies=None,
):
    """Download a thumbnail for an ee.Image.

    Args:
        ee_object (object): The ee.Image instance.
        out_img (str): The output file path to the png thumbnail.
        vis_params (dict): The visualization parameters.
        dimensions (int, optional):(a number or pair of numbers in format WIDTHxHEIGHT) Maximum dimensions of the thumbnail to render, in pixels. If only one number is passed, it is used as the maximum, and the other dimension is computed by proportional scaling. Defaults to 500.
        region (object, optional): Geospatial region of the image to render, it may be an ee.Geometry, GeoJSON, or an array of lat/lon points (E,S,W,N). If not set the default is the bounds image. Defaults to None.
        format (str, optional): Either 'png' or 'jpg'. Default to 'jpg'.
        timeout (int, optional): The number of seconds after which the request will be terminated. Defaults to 300.
        proxies (dict, optional): A dictionary of proxy servers to use for the request. Defaults to None.
    """

    if not isinstance(ee_object, ee.Image):
        raise TypeError("The ee_object must be an ee.Image.")

    ext = os.path.splitext(out_img)[1][1:]
    if ext not in ["png", "jpg"]:
        raise ValueError("The output image format must be png or jpg.")
    else:
        format = ext

    out_image = os.path.abspath(out_img)
    out_dir = os.path.dirname(out_image)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    if region is not None:
        vis_params["region"] = region

    vis_params["dimensions"] = dimensions
    vis_params["format"] = format
    vis_params["crs"] = crs
    url = ee_object.getThumbURL(vis_params)

    try:
        r = requests.get(url, stream=True, timeout=timeout, proxies=proxies)
    except Exception as e:
        print("An error occurred while downloading.")
        print(e)

    if r.status_code != 200:
        print("An error occurred while downloading.")
        print(r.json()["error"]["message"])

    else:
        with open(out_img, "wb") as fd:
            for chunk in r.iter_content(chunk_size=1024):
                fd.write(chunk)


def get_image_collection_thumbnails(
    ee_object,
    out_dir,
    vis_params,
    dimensions=500,
    region=None,
    format="jpg",
    names=None,
    verbose=True,
    timeout=300,
    proxies=None,
):
    """Download thumbnails for all images in an ImageCollection.

    Args:
        ee_object (object): The ee.ImageCollection instance.
        out_dir ([str): The output directory to store thumbnails.
        vis_params (dict): The visualization parameters.
        dimensions (int, optional):(a number or pair of numbers in format WIDTHxHEIGHT) Maximum dimensions of the thumbnail to render, in pixels. If only one number is passed, it is used as the maximum, and the other dimension is computed by proportional scaling. Defaults to 500.
        region (object, optional): Geospatial region of the image to render, it may be an ee.Geometry, GeoJSON, or an array of lat/lon points (E,S,W,N). If not set the default is the bounds image. Defaults to None.
        format (str, optional): Either 'png' or 'jpg'. Default to 'jpg'.
        names (list, optional): The list of output file names. Defaults to None.
        verbose (bool, optional): Whether or not to print hints. Defaults to True.
        timeout (int, optional): The number of seconds after which the request will be terminated. Defaults to 300.
        proxies (dict, optional): A dictionary of proxy servers to use for the request. Defaults to None.
    """
    if not isinstance(ee_object, ee.ImageCollection):
        print("The ee_object must be an ee.ImageCollection.")
        raise TypeError("The ee_object must be an ee.Image.")

    if format not in ["png", "jpg"]:
        raise ValueError("The output image format must be png or jpg.")

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    try:
        count = int(ee_object.size().getInfo())
        if verbose:
            print(f"Total number of images: {count}\n")

        if (names is not None) and (len(names) != count):
            print("The number of names is not equal to the number of images.")
            return

        if names is None:
            names = ee_object.aggregate_array("system:index").getInfo()

        images = ee_object.toList(count)

        for i in range(0, count):
            ######## ADDED THIS HERE SO TO CLIP THE IMAGE AND CENTRE ON LOCATION OF INTEREST######
            if ee.Image(images.get(i)).getString('Satellite').getInfo() == 'SENTINEL-1A' or ee.Image(images.get(i)).getString('Satellite').getInfo() == 'SENTINEL-1B':
              dist = 0.05
              roi = ee.Geometry.BBox(Lon.iloc[0] - dist, Lat.iloc[0]- dist,Lon.iloc[0]+ dist, Lat.iloc[0] +dist)
            else:
              dist = 0.05
              roi = ee.Geometry.BBox(Lon.iloc[0] - dist, Lat.iloc[0]- dist,Lon.iloc[0]+ dist, Lat.iloc[0] +dist)
            #######################################################################################
            image = ee.Image(images.get(i)).clip(roi) ####### HLC ADDED CLIP HERE
            name = str(names[i])
            ext = os.path.splitext(name)[1][1:]
            if ext != format:
                name = name + "." + format
            out_img = os.path.join(out_dir, name)
            if verbose:
                print(f"Downloading {i+1}/{count}: {name} ...")
            if ee.Image(image).getString('Satellite').getInfo()  == 'SENTINEL-1A' or ee.Image(image).getString('Satellite').getInfo()  == 'SENTINEL-1B': # If S1 use below parmaeters
              vis_params = {'bands':'VV', 'min':-25, 'max':5}                                                   # Display setting for the VV band
            else: vis_params = {'min': 0,'max': 3000,'bands': ['B4','B3','B2']}                                # Otherwise select the RGB bands
            get_image_thumbnail(
                image,
                out_img,
                vis_params,
                dimensions,
                region,
                format,
                timeout=timeout,
                proxies=proxies,
            )

    except Exception as e:
        print(e)




In [ ]:
def TimeLapse(out_dir = '/content/drive/MyDrive/Images'):
  print('TimeLapse Running')
  global Lat, Lon
  Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.4.xls')                          # Load in the wreck list
  Lat = pd.to_numeric(Wrecks.loc[Wrecks['Wreck_ID'] == WRKdropdown.value]['Latitude'])              # Get the latitude of selected wreck
  Lon = Wrecks.loc[Wrecks['Wreck_ID'] == WRKdropdown.value]['Longitude']                            # Get the longitude of selected wreck
  geom = ee.Geometry.Point(Lon.iloc[0],Lat.iloc[0]);                                                # Loction of chosen wreck

  ## Get the imageCollection, s1, s2 or both then combine
  if SatelliteNo.value == 'SENTINEL-1' or SatelliteNo.value == 'BOTH':                              # If to see if create feature collection with S1 or both
    S1ImgCol = (ee.ImageCollection('COPERNICUS/S1_GRD').                                            # Selects the Sentinel 1 image collection
      filterDate(str(StartDate.value), str(EndDate.value)).                                         # Selects only the dates from time period chosen above
      filterMetadata('instrumentMode', 'equals', 'IW').                                             # Selects the instrument mode that we want
      filter(ee.Filter.eq('orbitProperties_pass', str(orbit.value))).                               # Selects the orbit path we want
      filterBounds(geom))                                                                           # Selects only images that our wreck is contained within
    S1ImgCol = S1ImgCol.filter(ee.Filter.listContains('system:band_names', 'VV'))                   # Only selects VV band as missing in some images
    S1ImgCol = S1ImgCol.map(add_S1_date)                                                            # Adds the image date to image metadata in easy way to read

  if SatelliteNo.value == 'SENTINEL-2' or SatelliteNo.value == 'BOTH':                              # If to see if create feature collection with S2 or both
   S2ImgCol = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED').                                      # Selects the Sentinel 2 image collection
     filterDate(str(StartDate.value), str(EndDate.value)).                                          # Selects only the dates from time period chosen above
     filterBounds(geom).                                                                            # Selects only images that our wreck is contained within
     filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', max_cloud_cover.value))                 # Filter image collection by cloud cover
    ## Method of skipping duplicate images
   ImgList = ee.ImageCollection(S2ImgCol).toList(99999)                                             # Get the imageCollection as a list
   date_string = '1066-01-01'                                                                       # Store the previous date, use really old date here at start
   count = int(S2ImgCol.size().getInfo())                                                           # Get number of images
   for i in range(0, count):                                                                        # Loop through the images
     PrevDate = date_string                                                                         # Store previous date
     img = ee.Image(ee.List(ImgList).get(i))                                                        # Get the next image in the list
     IMgdate = img.date()                                                                           # Get the acquisition date of the image
     date_string = IMgdate.format('YYYY-MM-dd').getInfo()                                           # Format the date as a string
     if date_string == PrevDate:                                                                    # If image dates the same
       image_id = ee.Image(img).getString('system:index').getInfo()                                 # Get new image date
       S2ImgCol = S2ImgCol.filter(ee.Filter.neq('system:index', image_id))                          # Remove the duplicate
   S2ImgCol = S2ImgCol.map(add_S2_date)                                                             # Add the image date to the metadata in an easy way to read

  if SatelliteNo.value == 'BOTH':                                                                   # If both S1 and S2 combine collections
   ImageCol = S1ImgCol.merge(S2ImgCol)                                                              # Merge ImageCollections, note raw S1
   ImageCol = ImageCol.sort("Date")                                                                 # Order images by Date
  elif SatelliteNo.value == 'SENTINEL-1':                                                           # Create collections based just on S1
   ImageCol = S1ImgCol                                                                              # ImageCollection = S1
  elif  SatelliteNo.value == 'SENTINEL-2':                                                          # Create collections based just on S2
   ImageCol = S2ImgCol                                                                              # ImageCollection = S2

  ImgList = ee.ImageCollection(ImageCol).toList(99999)                                              # New list after removing of duplicates

  ## DOWNLOADS THE JPEGS #####
  out_mp4 = os.path.abspath(f"{WRKdropdown.value}.mp4")                                             # Path to save the gif
  img_params = {'min': 0,'max': 3000,'bands': ['B4','B3','B2']}                                     # Otherwise select the RGB bands
  count = ImageCol.size().getInfo()                                                                 # New count with replicates removed
  names = [os.path.join( out_dir, f"{WRKdropdown.value}_{str(i+1).zfill(int(len(str(count))))}.jpg")# List of image names to save adds a number.
         for i in range(count)]                                                                     # Loop for the number of names required
  get_image_collection_thumbnails(                                                                  # Function adapted from geeMap
    ImageCol,                                                                                       # Image collection
    out_dir,                                                                                        # Where to save
    vis_params=img_params,                                                                          # Image paramters, placeholder changed in next script based on satellite
  dimensions=resolution.value,                                                                      # Dimensions = resolution
  names=names)                                                                                      # The list of images

  from PIL import Image, ImageDraw, ImageFont
# define the path to the directory containing the images
  image_files = [os.path.join(out_dir, file) for file in os.listdir(out_dir) if file.endswith('.jpg')]

# find the size of the largest image
  max_width, max_height = 0, 0
  for image_file in image_files:
    with Image.open(image_file) as image:
        width, height = image.size
        max_width = max(max_width, width)
        max_height = max(max_height, height)

# resize all smaller images to the size of the largest image
  for image_file in image_files:
    with Image.open(image_file) as image:
        width, height = image.size
        if width < max_width or height < max_height:
            resized_image = image.resize((max_width, max_height))
            resized_image.save(image_file)

  ## The MP4 creation misses the last image so create a copy of it so included
  import shutil                                                                                     # Package used to copy
  src_file = os.path.join( out_dir, f"{WRKdropdown.value}_{count}.jpg")                             # Image to copy
  dst_file = f'{WRKdropdown.value}_last.jpg'                                                        # Name of new image
  shutil.copy(src_file, dst_file)                                                                   # create a copy of thmage
  files = os.listdir()                                                                              # List all the files in \content\
  jpg_files = [file for file in files if file.endswith('.jpg')]                                     # New list of all the jpg files
  jpg_files.sort()                                                                                  # Sort so in date order


  ## Adds the title, date and the point of the wreck to the map ##
                                                    # Packages for image manipulation
  from matplotlib import font_manager                                                               # To get font to write on images using PIL
  font = font_manager.FontProperties(family='sans-serif', weight='bold')                            # Need to download the font if not already got it
  file = font_manager.findfont(font)                                                                # Load the font
  font = ImageFont.truetype(file, 60)                                                               # Set up parameters of the font
  TitlePosition = (10, 10)                                                                          # Set the position and text for the title (first line)
  TitleText = f"{WRKdropdown.value}"                                                                # Title text
  DatePosition = (10, 10 + 100)                                                                     # Set the position and text for the second line
  for i in range(0, count):                                                                         # Loop through the jpgs
    filename =os.path.join(names[i])                                                                # gets the name of the first image
    img = ee.Image(ee.List(ImgList).get(i))                                                         # Get the next image in the list
    IMgdate = img.date()                                                                            # Get the acquisition date of the image
    date_string = IMgdate.format('YYYY-MM-dd').getInfo()                                            # Format the date as a string
    image = Image.open(filename)                                                                    # Open the image
    image = image.convert('RGB')                                                                    # Convert images to RGB otherwise the S1 images text and dot greyscale
    draw = ImageDraw.Draw(image)                                                                    # Create an ImageDraw object
    width, height = image.size                                                                      # Get the dimensions of the image
    x, y = width // 2, height // 2                                                                  # Calculate the coordinates of the center of the image
    draw.ellipse((x - 10, y - 10, x + 10, y + 10), fill='red')                                      # Draw a red dot at the center of the image
    draw.text(TitlePosition, TitleText, font=font, fill='red')                                      # Draw the first line of text (title) on the image
    draw.text(DatePosition, date_string, font=font, fill='red')                                     # Draw the second line of text (date) on the image
    image.save(filename)                                                                            # Save the image

  ## Create mp4 timelapse
  import imageio                                                                                    # Package to write mp4
  os.chdir(out_dir)                                                                                 # Doesnt like to use folder paths for some reason
  frames = [Image.open(image) for image in jpg_files]                                               # Open the jpgs
  imageio.mimwrite(out_mp4, frames, fps=0.5, quality=10,  codec='mjpeg')                            # Write the mp4 to \content\
  for image in jpg_files:                                                                           # Loop through through the jpgs
    os.remove(image)                                                                                # Remove the jpgs
  print('Timelapse created')                                                                        # Print completed
  os.chdir('/content')                                                                                # Return to G drive top

### 7.1 timelapse widgets

In [ ]:
def CreateTimeLapse():
  global FPS, resolution
  TLbutton = widgets.Button(description="Run TimeLapse")

 # Define a function to be called when the button is clicked
  def on_button_click(b):
    TimeLapse(out_dir = '/content/drive/MyDrive/Images')

 # Register the on_button_click function to be called when the button is clicked
  TLbutton.on_click(on_button_click)

  FPS = widgets.FloatSlider(value = 0.25,                                                  # Default value
                                min = 0.1,                                                      # Minimum value
                                max = 1.0,                                                    # Maximum value
                                step = .05,                                                     # Incremental step
                                description = 'FPS',                                   # Description of the slider
                                readout = True,                                               # Provides the value on screen
                                orientation = 'horizontal',                                   # Can be horizontal or vertical
                                layout=widgets.Layout(width='400px'))                         # Size of widget

  resolution = widgets.IntSlider(value = 2700,                                                  # Default value
                                      min = 500,                                                      # Minimum value
                                      max = 3000,                                                    # Maximum value
                                      step = 50,                                                     # Incremental step
                                      description = 'Resolution',                                   # Description of the slider
                                      readout = True,                                               # Provides the value on screen
                                      orientation = 'horizontal',                                   # Can be horizontal or vertical
                                      layout=widgets.Layout(width='400px'))                         # Size of widget

  # Display the button
  display(FPS,resolution, TLbutton)